## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [0]:
# File location and type
file_location = "/FileStore/tables/tips.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "false"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.csv(file_location, header=True, inferSchema = True)
df.show()

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|
|     18.43| 3.0|  Male|    No|Sun|Dinner|   4|
|     14.83|3.02|Female|    No|Sun|Dinner|   2|
|     21.58|3.92|  Male|    No|Sun|Dinner|   2|
|     10.33|1.67|Female|    No|Sun|Dinner|   3|
|     16.29|3.71|  Male|    No|Sun|Dinne

In [0]:
df.printSchema()

root
 |-- total_bill: double (nullable = true)
 |-- tip: double (nullable = true)
 |-- sex: string (nullable = true)
 |-- smoker: string (nullable = true)
 |-- day: string (nullable = true)
 |-- time: string (nullable = true)
 |-- size: integer (nullable = true)



In [0]:
df.columns

Out[3]: ['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size']

In [0]:
# Handling Categorical Features
from pyspark.ml.feature import StringIndexer

In [0]:
indexer = StringIndexer(inputCol="sex", outputCol="sex_indexed")
df_r = indexer.fit(df).transform(df)
df_r.show()

+----------+----+------+------+---+------+----+-----------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_indexed|
+----------+----+------+------+---+------+----+-----------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        1.0|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|        0.0|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|        0.0|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|        0.0|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|        1.0|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|        0.0|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|        0.0|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|        0.0|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|        0.0|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|        0.0|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|        0.0|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|        1.0|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|        0.0|
|     18.43| 3.0|  Male|    No|Sun|Dinne

In [0]:
indexer = StringIndexer(inputCols=["smoker", "day", "time"], outputCols=["smoker_indexed", "day_indexed", "time_indexed"])
df_r = indexer.fit(df_r).transform(df_r)
df_r.show()

+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_indexed|smoker_indexed|day_indexed|time_indexed|
+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        1.0|           0.0|        1.0|         0.0|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|         0.0|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|        1.0|           0.0|        1.0|         0.0|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|        0.0|           0.0|        1.0|         0.0|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|         0.0|


In [0]:
from pyspark.ml.feature import VectorAssembler
featureAssembler = VectorAssembler(inputCols =["tip", "sex_indexed", "smoker_indexed", "day_indexed", "time_indexed"], outputCol = "Independent Features")
output = featureAssembler.transform(df_r)

In [0]:
output.select("independent Features").show()

+--------------------+
|independent Features|
+--------------------+
|[1.01,1.0,0.0,1.0...|
|(5,[0,3],[1.66,1.0])|
| (5,[0,3],[3.5,1.0])|
|(5,[0,3],[3.31,1.0])|
|[3.61,1.0,0.0,1.0...|
|(5,[0,3],[4.71,1.0])|
| (5,[0,3],[2.0,1.0])|
|(5,[0,3],[3.12,1.0])|
|(5,[0,3],[1.96,1.0])|
|(5,[0,3],[3.23,1.0])|
|(5,[0,3],[1.71,1.0])|
|[5.0,1.0,0.0,1.0,...|
|(5,[0,3],[1.57,1.0])|
| (5,[0,3],[3.0,1.0])|
|[3.02,1.0,0.0,1.0...|
|(5,[0,3],[3.92,1.0])|
|[1.67,1.0,0.0,1.0...|
|(5,[0,3],[3.71,1.0])|
|[3.5,1.0,0.0,1.0,...|
|      (5,[0],[3.35])|
+--------------------+
only showing top 20 rows



In [0]:
finalized_data = output.select("Independent Features", "total_bill")

In [0]:
finalized_data.show()

+--------------------+----------+
|Independent Features|total_bill|
+--------------------+----------+
|[1.01,1.0,0.0,1.0...|     16.99|
|(5,[0,3],[1.66,1.0])|     10.34|
| (5,[0,3],[3.5,1.0])|     21.01|
|(5,[0,3],[3.31,1.0])|     23.68|
|[3.61,1.0,0.0,1.0...|     24.59|
|(5,[0,3],[4.71,1.0])|     25.29|
| (5,[0,3],[2.0,1.0])|      8.77|
|(5,[0,3],[3.12,1.0])|     26.88|
|(5,[0,3],[1.96,1.0])|     15.04|
|(5,[0,3],[3.23,1.0])|     14.78|
|(5,[0,3],[1.71,1.0])|     10.27|
|[5.0,1.0,0.0,1.0,...|     35.26|
|(5,[0,3],[1.57,1.0])|     15.42|
| (5,[0,3],[3.0,1.0])|     18.43|
|[3.02,1.0,0.0,1.0...|     14.83|
|(5,[0,3],[3.92,1.0])|     21.58|
|[1.67,1.0,0.0,1.0...|     10.33|
|(5,[0,3],[3.71,1.0])|     16.29|
|[3.5,1.0,0.0,1.0,...|     16.97|
|      (5,[0],[3.35])|     20.65|
+--------------------+----------+
only showing top 20 rows



In [0]:
from pyspark.ml.regression import LinearRegression
#train test split
train_data, test_data = finalized_data.randomSplit([0.75,0.25])
regressor = LinearRegression(featuresCol ="Independent Features", labelCol="total_bill")
regressor = regressor.fit(train_data)

In [0]:
regressor.coefficients

Out[14]: DenseVector([4.0069, -1.2441, 1.4653, 0.4702, -3.1537])

In [0]:
regressor.intercept

Out[15]: 8.473798604755581

In [0]:
#Predictions
pred_results = regressor.evaluate(test_data)

In [0]:
pred_results.predictions.show()

/databricks/spark/python/pyspark/sql/context.py:134: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(
+--------------------+----------+------------------+
|Independent Features|total_bill|        prediction|
+--------------------+----------+------------------+
|      (5,[0],[1.25])|     10.51|  13.4823746977553|
|       (5,[0],[2.0])|     12.69|16.487520353555134|
|      (5,[0],[2.64])|     17.59| 19.05191131317099|
|      (5,[0],[3.15])|     20.08|21.095410359114876|
|      (5,[0],[3.39])|     11.61|22.057056968970823|
|      (5,[0],[4.08])|     17.92|24.821790972306665|
| (5,[0,1],[2.2,1.0])|     14.73|  16.0447675213533|
|(5,[0,1],[2.45,1.0])|     20.69|17.046482739953248|
| (5,[0,1],[3.0,1.0])|     15.06| 19.25025622087312|
| (5,[0,2],[1.5,1.0])|     11.59|15.949409646503934|
|(5,[0,2],[1.64,1.0])|     15.36|16.510370168919906|
|(5,[0,2],[1.98,1.0])|     11.02| 17.87270286621583|
| (5,[0,2],[2.0,1.0])|     13.81|17.952840083703823

In [0]:
#Performance Metric 
pred_results.r2, pred_results.meanAbsoluteError, pred_results.meanSquaredError

Out[20]: (0.4984538935559052, 4.894571137887134, 41.68491469728296)